In [6]:
import importlib
# --------------------
from sklearn.utils import gen_batches
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
import numpy as np
from numpy.linalg import norm,inv,matrix_rank

import h5py
from sklearn.cross_decomposition import PLSRegression, PLSSVD

# import warnings
# warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [7]:
def RollingCV(tscv,X):
    cv = tscv.split(X)
    (train_index, test_index) = next(cv)
    yield(
        train_index,
        test_index
    )
    test_size = len(test_index)

    for (train_index, test_index) in (cv):
        yield(
            train_index[-test_size:],
            test_index
        )

def Comp_Model_Score(estimator, cv, X, Y):
    score = np.zeros((n_fold,))
    for i,(train_index, test_index) in enumerate(cv):
        try:
            estimator.fit(X[train_index], Y[train_index])
            y_true = Y[test_index]
            y_pred = estimator.predict(X[test_index])
            score[i] = RMSE(y_true, y_pred)
        except:
            score[i] = np.nan
            break
    return np.mean(score)

In [8]:
tmp = h5py.File('./data/TW_PM25.h5', 'r')
X_train, Y_train = tmp['X_train'], tmp['Y_train']
n_train = X_train.shape[0]

X_train = X_train[0:n_train]
Y_train = Y_train[0:n_train]
tmp.close()



In [9]:
print(X_train.shape)
print(Y_train.shape)
n_fold = 17
test_size=30
tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break
    
print("="*40)
for i, (train_index, test_index) in enumerate(RollingCV(tscv,X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break    

(877, 1800)
(877, 1800)
Fold 0:
  Train: len=367
  Test:  len=30
Fold 1:
  Train: len=397
  Test:  len=30
Fold 2:
  Train: len=427
  Test:  len=30
Fold 3:
  Train: len=457
  Test:  len=30
Fold 4:
  Train: len=487
  Test:  len=30
  Train: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187

In [12]:
n_comp_lst=list(range(1,16))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(PLSRegression(scale=False), parameters,
                   cv=tscv, n_jobs=4,
                   scoring='neg_root_mean_squared_error')
print(clf)
tim.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tim.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=17, test_size=30),
             estimator=PLSRegression(scale=False), n_jobs=4,
             param_grid={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15]},
             scoring='neg_root_mean_squared_error')


CV elapsed time: 42.97s
best parameter:  {'n_components': 10} ; score:  -7.1548572e+00


In [5]:
import Code.SVDPLS
importlib.reload(Code.SVDPLS)
from Code.SVDPLS import SVDPLS

n_comp_lst=list(range(1,16))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(SVDPLS(scale=False), parameters,
                   cv=tscv, n_jobs=4, verbose=3,  
                   scoring='neg_root_mean_squared_error')
print(clf)
tim.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tim.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=17, test_size=30),
             estimator=SVDPLS(scale=False), n_jobs=4,
             param_grid={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15]},
             scoring='neg_root_mean_squared_error', verbose=3)
Fitting 17 folds for each of 15 candidates, totalling 255 fits


CV elapsed time: 381.18s
best parameter:  {'n_components': 3} ; score:  -7.6303875e+00


[CV 1/17] END ..................n_components=1;, score=-5.015 total time=   5.9s
[CV 5/17] END ..................n_components=1;, score=-8.554 total time=   5.9s
[CV 9/17] END ..................n_components=1;, score=-9.067 total time=   5.9s
[CV 14/17] END .................n_components=1;, score=-6.029 total time=   6.1s
[CV 1/17] END ..................n_components=2;, score=-4.951 total time=   5.9s
[CV 5/17] END ..................n_components=2;, score=-8.112 total time=   5.9s
[CV 8/17] END ..................n_components=2;, score=-8.694 total time=   6.0s
[CV 12/17] END .................n_components=2;, score=-7.427 total time=   6.1s
[CV 16/17] END .................n_components=2;, score=-8.430 total time=   6.2s
[CV 6/17] END ..................n_components=3;, score=-8.436 total time=   6.0s
[CV 10/17] END ................n_components=3;, score=-10.126 total time=   5.9s
[CV 14/17] END .................n_components=3;, score=-6.600 total time=   6.0s
[CV 1/17] END ..............

[CV 3/17] END ..................n_components=1;, score=-6.520 total time=   6.0s
[CV 8/17] END ..................n_components=1;, score=-9.483 total time=   6.0s
[CV 12/17] END .................n_components=1;, score=-7.450 total time=   6.0s
[CV 16/17] END .................n_components=1;, score=-8.143 total time=   6.0s
[CV 3/17] END ..................n_components=2;, score=-6.477 total time=   5.8s
[CV 6/17] END ..................n_components=2;, score=-8.232 total time=   5.9s
[CV 9/17] END ..................n_components=2;, score=-8.191 total time=   6.0s
[CV 13/17] END .................n_components=2;, score=-5.242 total time=   6.0s
[CV 17/17] END .................n_components=2;, score=-7.790 total time=   6.1s
[CV 5/17] END ..................n_components=3;, score=-7.797 total time=   5.9s
[CV 9/17] END ..................n_components=3;, score=-8.122 total time=   6.0s
[CV 13/17] END .................n_components=3;, score=-5.354 total time=   5.9s
[CV 16/17] END .............

[CV 2/17] END ..................n_components=1;, score=-6.529 total time=   5.9s
[CV 6/17] END ..................n_components=1;, score=-8.381 total time=   5.9s
[CV 10/17] END ................n_components=1;, score=-10.709 total time=   5.9s
[CV 13/17] END .................n_components=1;, score=-5.035 total time=   5.9s
[CV 17/17] END .................n_components=1;, score=-7.953 total time=   6.1s
[CV 4/17] END ..................n_components=2;, score=-9.228 total time=   6.1s
[CV 11/17] END .................n_components=2;, score=-7.719 total time=   6.0s
[CV 14/17] END .................n_components=2;, score=-6.510 total time=   6.1s
[CV 1/17] END ..................n_components=3;, score=-4.991 total time=   5.9s
[CV 3/17] END ..................n_components=3;, score=-6.296 total time=   5.9s
[CV 7/17] END ..................n_components=3;, score=-9.730 total time=   5.9s
[CV 11/17] END .................n_components=3;, score=-7.794 total time=   6.0s
[CV 15/17] END .............

In [16]:
import Code.SIMPLS
importlib.reload(Code.SIMPLS)
from Code.SIMPLS import SIMPLS

n_comp_lst=list(range(1,16))
parameters = {'n_components':n_comp_lst}
clf = GridSearchCV(SIMPLS(),  parameters,
                   cv=tscv, n_jobs=4, verbose=3,  
                   scoring='neg_root_mean_squared_error')
print(clf)
tim.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tim.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=17, test_size=30),
             estimator=SIMPLS(), n_jobs=4,
             param_grid={'n_components': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                          13, 14, 15]},
             scoring='neg_root_mean_squared_error', verbose=3)
Fitting 17 folds for each of 15 candidates, totalling 255 fits


CV elapsed time: 127.47s
best parameter:  {'n_components': 11} ; score:  -7.1476354e+00


In [15]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

scores = []
params = []
n_comp_list=list(range(1,16))

tim_tot.tic()
for n_comp in n_comp_list:
    tim.tic()
    pls=ISIMPLS(n_components=n_comp)
    scores.append(Comp_Model_Score(pls, RollingCV(tscv,X_train), X_train, Y_train) )
    params.append({'n_components': n_comp})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")


params={'n_components': 1}, score=7.7670599e+00, elapsed time=3.9s


params={'n_components': 2}, score=7.5039548e+00, elapsed time=5.8s


params={'n_components': 3}, score=7.3682301e+00, elapsed time=6.7s


params={'n_components': 4}, score=7.3129378e+00, elapsed time=8.0s


params={'n_components': 5}, score=7.3165210e+00, elapsed time=10.7s


...

params={'n_components': 11}, score=7.1476354e+00, elapsed time=19.4s


params={'n_components': 12}, score=7.1902018e+00, elapsed time=21.0s


params={'n_components': 13}, score=7.1989453e+00, elapsed time=23.3s


params={'n_components': 14}, score=7.2195319e+00, elapsed time=25.5s


params={'n_components': 15}, score=7.2495417e+00, elapsed time=26.3s

best parameter: {'n_components': 11}; score: 7.1476354e+00; total time=222.2s


In [21]:
import Code.OLPLS
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS
# with t = np.dot(x, self.W[:, 0])
scores = []
params = []
mu_list= [1e-5, 1e-6, 1e-7, 1e-8]
amnesic_list= [1e-2] + list(np.arange(0.1,1.0,0.1)) + [0.99]
print(f"mu: {mu_list}")
print(f"amnesic: {amnesic_list}")

tim_tot.tic()
for mu in mu_list:
    for amnesic in amnesic_list:
        tim.tic()
        pls=OLPLS(n_components=10, mu=mu, amnesic=amnesic)
        scores.append(Comp_Model_Score(pls, RollingCV(tscv,X_train), X_train, Y_train) )
        params.append({'mu': mu, 'amnesic': amnesic})
        print(f"params={params[-1]}, score={scores[-1]:.7e}, "
              +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}, "
      +f"total time={tim_tot.tocvalue():.1f}s")

mu: [1e-05, 1e-06, 1e-07, 1e-08]
amnesic: [0.01, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6, 0.7000000000000001, 0.8, 0.9, 0.99]


params={'mu': 1e-05, 'amnesic': 0.01}, score=nan, elapsed time=25.3s


params={'mu': 1e-05, 'amnesic': 0.1}, score=nan, elapsed time=25.3s


params={'mu': 1e-05, 'amnesic': 0.2}, score=nan, elapsed time=25.3s


params={'mu': 1e-05, 'amnesic': 0.30000000000000004}, score=nan, elapsed time=25.3s


...

params={'mu': 1e-08, 'amnesic': 0.6}, score=nan, elapsed time=25.3s


params={'mu': 1e-08, 'amnesic': 0.7000000000000001}, score=nan, elapsed time=25.3s


params={'mu': 1e-08, 'amnesic': 0.8}, score=nan, elapsed time=45.9s


params={'mu': 1e-08, 'amnesic': 0.9}, score=9.0979664e+00, elapsed time=58.2s


params={'mu': 1e-08, 'amnesic': 0.99}, score=9.0464381e+00, elapsed time=58.2s

best parameter: {'mu': 1e-08, 'amnesic': 0.99}; score: 9.0464381e+00, total time=1219.2s


In [13]:
import Code.OLPLS
importlib.reload(Code.OLPLS)
from Code.OLPLS import OLPLS
# with t = np.dot(x, self.W[:, 0])
scores = []
params = []
mu_list= [1e-8, 1e-9]
amnesic_list= [1e-2] + list(np.arange(0.1,1.0,0.1)) + [0.99]
print(f"mu: {mu_list}")
print(f"amnesic: {amnesic_list}")

tim_tot.tic()
for mu in mu_list:
    for amnesic in amnesic_list:
        tim.tic()
        pls=OLPLS(n_components=10, mu=mu, amnesic=amnesic)
        scores.append(Comp_Model_Score(pls, RollingCV(tscv,X_train), X_train, Y_train) )
        params.append({'mu': mu, 'amnesic': amnesic})
        print(f"params={params[-1]}, score={scores[-1]:.7e}, "
              +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}, "
      +f"total time={tim_tot.tocvalue():.1f}s")

mu: [1e-08, 1e-09]
amnesic: [0.01, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6, 0.7000000000000001, 0.8, 0.9, 0.99]


params={'mu': 1e-08, 'amnesic': 0.01}, score=nan, elapsed time=25.1s


params={'mu': 1e-08, 'amnesic': 0.1}, score=nan, elapsed time=25.0s


params={'mu': 1e-08, 'amnesic': 0.2}, score=nan, elapsed time=25.1s


params={'mu': 1e-08, 'amnesic': 0.30000000000000004}, score=nan, elapsed time=24.7s


...

params={'mu': 1e-09, 'amnesic': 0.6}, score=9.4626534e+00, elapsed time=57.0s


params={'mu': 1e-09, 'amnesic': 0.7000000000000001}, score=9.4621936e+00, elapsed time=57.0s


params={'mu': 1e-09, 'amnesic': 0.8}, score=9.4616873e+00, elapsed time=57.0s


params={'mu': 1e-09, 'amnesic': 0.9}, score=9.4610525e+00, elapsed time=57.1s


params={'mu': 1e-09, 'amnesic': 0.99}, score=9.4575165e+00, elapsed time=57.0s

best parameter: {'mu': 1e-08, 'amnesic': 0.99}; score: 9.0463859e+00, total time=946.8s


In [10]:
import Code.SGDPLS
importlib.reload(Code.SGDPLS)
from Code.SGDPLS import SGDPLS

scores = []
params = []
eta_list= [1e-4, 1e-5, 1e-6, 1e-7]
print(f"eta: {eta_list}")

tim_tot.tic()
for eta in eta_list:
    tim.tic()
    pls=SGDPLS(n_components=10, eta=eta)
    scores.append(Comp_Model_Score(pls, RollingCV(tscv,X_train), X_train, Y_train) )
    params.append({'eta': eta})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}, "
      +f"total time={tim_tot.tocvalue():.1f}s")


eta: [0.0001, 1e-05, 1e-06, 1e-07]


params={'eta': 0.0001}, score=nan, elapsed time=1178.1s


params={'eta': 1e-05}, score=nan, elapsed time=1172.8s


params={'eta': 1e-06}, score=nan, elapsed time=1172.7s


params={'eta': 1e-07}, score=nan, elapsed time=1174.6s


ValueError: All-NaN slice encountered

In [11]:
import Code.SGDPLS
importlib.reload(Code.SGDPLS)
from Code.SGDPLS import SGDPLS

scores = []
params = []
eta_list= [1e-8, 1e-9]
print(f"eta: {eta_list}")

tim_tot.tic()
for eta in eta_list:
    tim.tic()
    pls=SGDPLS(n_components=10, eta=eta)
    scores.append(Comp_Model_Score(pls, RollingCV(tscv,X_train), X_train, Y_train) )
    params.append({'eta': eta})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}, "
      +f"total time={tim_tot.tocvalue():.1f}s")


eta: [1e-08, 1e-09]


params={'eta': 1e-08}, score=nan, elapsed time=1192.3s


params={'eta': 1e-09}, score=nan, elapsed time=1183.9s


ValueError: All-NaN slice encountered

In [14]:
import Code.SGDPLS
importlib.reload(Code.SGDPLS)
from Code.SGDPLS import SGDPLS

eta_list= [1e-5, 1e-6, 1e-7]
parameters = {'eta': eta_list}
clf = GridSearchCV(SGDPLS(),  parameters,
                   cv=tscv, verbose=4, n_jobs=3, 
                   scoring='neg_root_mean_squared_error')
print(clf)
tim.tic()
clf.fit(X_train, Y_train)
print("CV elapsed time: {:.2f}s".format(tim.tocvalue()))
print("best parameter: ", clf.best_params_,
      "; score: {:15.7e}".format(clf.best_score_))


GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=17, test_size=30),
             estimator=SGDPLS(), n_jobs=3,
             param_grid={'eta': [1e-05, 1e-06, 1e-07]},
             scoring='neg_root_mean_squared_error', verbose=4)
Fitting 17 folds for each of 3 candidates, totalling 51 fits


/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:95: RuntimeWarning: overflow encountered in matmul
  self.x_rotations += self.eta * np.matmul(np.matmul(xi, yi.T), self.y_rotations)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:96: RuntimeWarning: overflow encountered in matmul
  self.y_rotations += self.eta * np.matmul(np.matmul(yi, xi.T), tmp)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:96: RuntimeWarning: invalid value encountered in matmul
  self.y_rotations += self.eta * np.matmul(np.matmul(yi, xi.T), tmp)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:95: RuntimeWarning: invalid value encountered in matmul
  self.x_rotations += self.eta * np.matmul(np.matmul(xi, yi.T), self.y_rotations)


/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:95: RuntimeWarning: overflow encountered in matmul
  self.x_rotations += self.eta * np.matmul(np.matmul(xi, yi.T), self.y_rotations)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:96: RuntimeWarning: overflow encountered in matmul
  self.y_rotations += self.eta * np.matmul(np.matmul(yi, xi.T), tmp)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:96: RuntimeWarning: invalid value encountered in matmul
  self.y_rotations += self.eta * np.matmul(np.matmul(yi, xi.T), tmp)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:95: RuntimeWarning: invalid value encountered in matmul
  self.x_rotations += self.eta * np.matmul(np.matmul(xi, yi.T), self.y_rotations)


/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:96: RuntimeWarning: overflow encountered in matmul
  self.y_rotations += self.eta * np.matmul(np.matmul(yi, xi.T), tmp)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:95: RuntimeWarning: overflow encountered in matmul
  self.x_rotations += self.eta * np.matmul(np.matmul(xi, yi.T), self.y_rotations)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:96: RuntimeWarning: invalid value encountered in matmul
  self.y_rotations += self.eta * np.matmul(np.matmul(yi, xi.T), tmp)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:95: RuntimeWarning: invalid value encountered in matmul
  self.x_rotations += self.eta * np.matmul(np.matmul(xi, yi.T), self.y_rotations)


/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:95: RuntimeWarning: overflow encountered in matmul
  self.x_rotations += self.eta * np.matmul(np.matmul(xi, yi.T), self.y_rotations)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:96: RuntimeWarning: overflow encountered in matmul
  self.y_rotations += self.eta * np.matmul(np.matmul(yi, xi.T), tmp)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:96: RuntimeWarning: invalid value encountered in matmul
  self.y_rotations += self.eta * np.matmul(np.matmul(yi, xi.T), tmp)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:95: RuntimeWarning: invalid value encountered in matmul
  self.x_rotations += self.eta * np.matmul(np.matmul(xi, yi.T), self.y_rotations)


...

/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:95: RuntimeWarning: overflow encountered in matmul
  self.x_rotations += self.eta * np.matmul(np.matmul(xi, yi.T), self.y_rotations)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:96: RuntimeWarning: overflow encountered in matmul
  self.y_rotations += self.eta * np.matmul(np.matmul(yi, xi.T), tmp)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:96: RuntimeWarning: invalid value encountered in matmul
  self.y_rotations += self.eta * np.matmul(np.matmul(yi, xi.T), tmp)
/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py:95: RuntimeWarning: invalid value encountered in matmul
  self.x_rotations += self.eta * np.matmul(np.matmul(xi, yi.T), self.y_rotations)


[CV 2/17] END ..........................eta=1e-05;, score=nan total time=28.2min
[CV 5/17] END ..........................eta=1e-05;, score=nan total time=38.7min
[CV 8/17] END ..........................eta=1e-05;, score=nan total time=44.6min
[CV 12/17] END .........................eta=1e-05;, score=nan total time=55.2min
[CV 15/17] END .........................eta=1e-05;, score=nan total time=55.7min
[CV 1/17] END ..........................eta=1e-06;, score=nan total time=26.0min
[CV 2/17] END ..........................eta=1e-06;, score=nan total time=28.0min
[CV 3/17] END ..........................eta=1e-06;, score=nan total time=30.0min
[CV 6/17] END ..........................eta=1e-06;, score=nan total time=37.5min
[CV 9/17] END ..........................eta=1e-06;, score=nan total time=44.9min
[CV 12/17] END .........................eta=1e-06;, score=nan total time=51.6min
[CV 15/17] END .........................eta=1e-06;, score=nan total time=63.3min
[CV 1/17] END ..............

[CV 3/17] END ..........................eta=1e-05;, score=nan total time=31.0min
[CV 6/17] END ..........................eta=1e-05;, score=nan total time=37.0min
[CV 9/17] END ..........................eta=1e-05;, score=nan total time=43.2min
[CV 11/17] END .........................eta=1e-05;, score=nan total time=47.1min
[CV 14/17] END .........................eta=1e-05;, score=nan total time=59.6min
[CV 17/17] END .........................eta=1e-05;, score=nan total time=68.1min
[CV 5/17] END ..........................eta=1e-06;, score=nan total time=38.5min
[CV 8/17] END ..........................eta=1e-06;, score=nan total time=43.2min
[CV 11/17] END .........................eta=1e-06;, score=nan total time=49.1min
[CV 14/17] END .........................eta=1e-06;, score=nan total time=54.8min
[CV 17/17] END .........................eta=1e-06;, score=nan total time=61.5min
[CV 4/17] END ..........................eta=1e-07;, score=nan total time=36.2min
[CV 7/17] END ..............

ValueError: 
All the 51 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
51 fits failed with the following error:
Traceback (most recent call last):
  File "/home/tingan/miniconda3/envs/pls/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py", line 98, in fit
    self._find_projections(X,Y)
  File "/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py", line 104, in _find_projections
    t = np.dot(X[i], gram_schmidt_normalize(self.x_rotations.T[c-1][:, np.newaxis]).ravel()  )
  File "/video/data/tingan/GitRep/IPLSreview/Code/SGDPLS.py", line 14, in gram_schmidt_normalize
    X = check_array(X, dtype=FLOAT_DTYPES, copy=copy, ensure_2d=

[CV 1/17] END ..........................eta=1e-05;, score=nan total time=26.1min
[CV 4/17] END ..........................eta=1e-05;, score=nan total time=36.1min
[CV 7/17] END ..........................eta=1e-05;, score=nan total time=43.6min
[CV 10/17] END .........................eta=1e-05;, score=nan total time=50.9min
[CV 13/17] END .........................eta=1e-05;, score=nan total time=57.2min
[CV 16/17] END .........................eta=1e-05;, score=nan total time=65.6min
[CV 4/17] END ..........................eta=1e-06;, score=nan total time=36.3min
[CV 7/17] END ..........................eta=1e-06;, score=nan total time=41.3min
[CV 10/17] END .........................eta=1e-06;, score=nan total time=45.7min
[CV 13/17] END .........................eta=1e-06;, score=nan total time=54.7min
[CV 16/17] END .........................eta=1e-06;, score=nan total time=67.0min
[CV 2/17] END ..........................eta=1e-07;, score=nan total time=31.7min
[CV 5/17] END ..............